In [3]:
# с семинара

capital = 100  # в тыс. руб.
step = 10
profit_of_investment = [[0, 7, 13, 20, 33, 35],
                        [0, 8, 12, 21, 28, 40],
                        [0, 10, 11, 19, 30, 34],
                        [0, 6, 9, 24, 30, 36]]

In [76]:
# мой вариант

capital = 700  # в тыс. руб.
step = 100
profit_of_investment = [[0, 28, 45, 65, 78, 90, 102, 113],
                        [0, 25, 41, 55, 65, 75, 80, 85],
                        [0, 15, 25, 40, 50, 62, 73, 82],
                        [0, 20, 33, 42, 48, 53, 56, 58]]

In [83]:
# my test case

capital = 50  # в тыс. руб.
step = 10
profit_of_investment = [[0, 1],
                        [0, 1, 2, 3],
                        [0, 1, 2]]

In [80]:
# Никита

capital = 700
step = 100
profit_of_investment = [
    [0, 3, 18, 59, 66, 64, 72, 81],
    [0, 8, 24, 26, 40, 42, 51, 57],
    [0, 6, 15, 17, 31, 33, 45, 67],
    [0, 9, 30, 45, 67, 70, 72, 74],
]

In [86]:
# Вариант 5

capital = 700
step = 100
profit_of_investment = [
    [0, 37, 64, 87, 105, 120, 134, 145],
    [0, 48, 75, 98, 120, 132, 144, 156],
    [0, 85, 90, 111, 118, 124, 129, 132],
    [0, 47, 70, 80, 86, 91, 94, 98]
]

In [92]:
def calc_possible_u(profit_of_investment, step):
    """
    Считаем возможные управления U_matrix. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :return: лист из листов возможных управлений
    """
    U_matrix = []
    for k in range(0, len(profit_of_investment)):  # идем по предприятиям
        max_u_index = len(profit_of_investment[k])
        U = [i * step for i in range(max_u_index)]  # заполняем строку матрицы
        U_matrix.append(U)  # добавляем строку в матрицу
    return U_matrix


def calc_possible_x(profit_of_investment, step):
    """
    Считаем возможные состояния X_matrix. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :return:
    """
    X_matrix = []
    U_matrix = calc_possible_u(profit_of_investment, step)
    x_initial = capital
    X_matrix.append([x_initial])
    for k in range(1, len(profit_of_investment)):  # идем по предприятиям
        # посчитаем все возможные состояния после того, как подействуем управлением u на k шаге на состояние x после k-1 шага
        X = []
        for x in X_matrix[k - 1]:
            for u in U_matrix[k]:
                X.append(x - u)
        X = list(set(X))  # удалим дубликаты
        X = sorted(X)  # отсортируем
        X = [x for x in X if x >= 0]  # уберем отрицательные значения
        X_matrix.append(X)
    return X_matrix


def print_for_find_maximum_profit(profit, paths):
    for i in range(1, len(paths) + 1):
        print("Variant {0}:".format(i))
        u = paths[i - 1]
        for k in range(0, len(u)):
            print("It should be spent {0} on company {1}".format(u[k], k + 1))
    print("Total profit is {0}".format(profit))


def find_maximum_profit(profit_matrix, capital):
    """
    :param profit_matrix: лист из листов с доходом от предприятий
    :param capital: начальный капитал
    :return: словарь, в котором хранится максимальный прибыли и массив из последовательнойтей управлений, которые приведут к максимальное прибыли
    """

    # заполняем переменные, которые понядобятся нам для вычисления
    U_matrix = calc_possible_u(profit_matrix, step)  # возможные управления
    X_matrix = calc_possible_x(profit_matrix, step)  # возможные состояния
    num_companies = len(profit_matrix)  # кол-во компаний
    F = {}  # хеш-таблица, в которой мы храним все значения функции Беллмана

    # идем от (num_companies - 1) до 0
    for k in range(num_companies - 1, -1, -1):

        # # заполняем лист возможных состояний и управлений
        X = [i for i in X_matrix[k]]  # заполняем лист возможных x (состояний)
        U = [i for i in U_matrix[k]]  # заполняем лист возможных u (управлений)

        #идем по возможным состояниям
        for x in X:

            # перебираем все возможные управления
            w_list = []
            for u in U:
                if u > x:  # если наше управление предполагает страты большие, чем у нас есть денег
                    break  # то значит такое управление невалидно => выходим из цикла
                if k == num_companies - 1:  # если мы на шаге, соответсвующему последней компании
                    w_list.append([profit_matrix[k][U.index(u)], u])  # сохраняем и значение F, и управление
                else:
                    w_list.append([profit_matrix[k][U.index(u)] + F[(k + 1, x - u)]["w"], u])

            # ищем максимальную прибыль
            w_max = 0  # максимальная прибыль
            for w, u in w_list:
                if w > w_max:
                    w_max = w
            u_max_list = []  # управления, при которых у нас достигается максимальная прибыль
            for w, u in w_list:  # сохраняем все возможные управления, соответствующие максимальное прибыли
                if w == w_max:
                    u_max_list.append(u)

            # добавляем значения функции Беллмана
            F[(k, x)] = {
                "w": w_max,
                "u": u_max_list
            }

    paths = [{  # лист из возможных управлений, которые привелут к максимальной прибыли
        "x": capital,
        "u": []
    }]
    for k in range(0, num_companies):  # обход графа в ширину
        new_paths = []
        for path in paths:
            x_of_path = path["x"]
            u_of_path = path["u"]
            U = F[(k, x_of_path)]["u"]
            for u in U:
                new_paths.append({
                    "x": x_of_path - u,
                    "u": [*u_of_path, u]
                })
        paths = new_paths
    paths = [path["u"] for path in paths]

    maximum_profit = {
        "profit": F[(0, capital)]["w"],
        "paths": paths
    }

    print_for_find_maximum_profit(maximum_profit["profit"], maximum_profit["paths"])

    return maximum_profit

In [93]:
maximum_profit = find_maximum_profit(profit_of_investment, capital)

Variant 1:
It should be spent 200 on company 1
It should be spent 200 on company 2
It should be spent 100 on company 3
It should be spent 200 on company 4
Variant 2:
It should be spent 200 on company 1
It should be spent 300 on company 2
It should be spent 100 on company 3
It should be spent 100 on company 4
Variant 3:
It should be spent 300 on company 1
It should be spent 200 on company 2
It should be spent 100 on company 3
It should be spent 100 on company 4
Total profit is 294
